<a href="https://colab.research.google.com/github/allenhu08/capstone/blob/main/Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!git clone https://github.com/allenhu08/capstone.git || cd capstone && git pull

fatal: destination path 'capstone' already exists and is not an empty directory.
Already up to date.


In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('./capstone/datasets/rawdata_new.csv')


In [36]:
print(df.columns.tolist())
print(df.head())

['id', 'date', 'total_agr_land', 'workforce', 'trees', 'arable_land', 'ley', 'perm_grass', 'perm_crops', 'prot_cult', 'add_agr_land', 'oth_land', 'qual_landscape', 'spec_crops', 'grapes', 'vegetables', 'fruits', 'oth_spec_crops', 'animal_unit', 'cattle_animal_unit', 'dairy_cows', 'suckler_cows', 'hor_shee_goa', 'pigs_poultry', 'pigs', 'poultry', 'eco_proof', 'organic', 'farmtype', 'region', 'age', 'educ', 'cons_general', 'percep_hail', 'percep_drought', 'percep_frost', 'percep_heavyrain', 'percep_longrain', 'percep_heat', 'cons_farm', 'self_reduc', 'self_act', 'self_cap', 'self_conf', 'self_not', 'legum', 'legum_eff', 'conc', 'conc_eff', 'add', 'add_eff', 'lact', 'lact_eff', 'breed', 'breed_eff', 'covman', 'covman_eff', 'comp', 'comp_eff', 'drag', 'drag_eff', 'cov', 'cov_eff', 'plough', 'plough_eff', 'solar', 'solar_eff', 'biog', 'biog_eff', 'ecodr', 'ecodr_eff', 'legum_imag', 'conc_imag', 'add_imag', 'lact_imag', 'breed_imag', 'covman_imag', 'comp_imag', 'drag_imag', 'cov_imag', 'plou

In [10]:
farm_info_cols = ['total_agr_land', 'workforce', 'trees', 'arable_land', 'ley', 'perm_grass', 'perm_crops', 'prot_cult', 'add_agr_land', 'oth_land', 'qual_landscape', 'eco_proof', 'organic',  'age'] #'farmtype', 'region',
crops_info_cols = ['spec_crops', 'grapes', 'vegetables', 'fruits', 'oth_spec_crops']
animal_info_cols = ['animal_unit', 'cattle_animal_unit', 'dairy_cows', 'suckler_cows', 'hor_shee_goa', 'pigs_poultry', 'pigs', 'poultry']
attitude_info_cols = ['educ', 'cons_general', 'percep_hail', 'percep_drought', 'percep_frost', 'percep_heavyrain', 'percep_longrain', 'percep_heat', 'cons_farm', 'self_reduc', 'self_act', 'self_cap', 'self_conf', 'self_not']
meansures_info_cols = ['legum', 'legum_eff', 'conc', 'conc_eff', 'add', 'add_eff', 'lact', 'lact_eff', 'breed', 'breed_eff', 'covman', 'covman_eff', 'comp', 'comp_eff', 'drag', 'drag_eff', 'cov', 'cov_eff', 'plough', 'plough_eff', 'solar', 'solar_eff', 'biog', 'biog_eff', 'ecodr', 'ecodr_eff']
measures_effect_info_cols = ['legum_eff', 'conc_eff', 'add_eff', 'lact_eff', 'breed_eff', 'covman_eff', 'comp_eff', 'drag_eff', 'cov_eff', 'plough_eff', 'solar_eff', 'biog_eff', 'ecodr_eff']

# df = df[farm_info_cols + crops_info_cols + animal_info_cols + meansures_info_cols]
effect_df = df[measures_effect_info_cols]
effect_df = effect_df.fillna(0)
effect_df.head()
effect_vals = effect_df.values



In [32]:
from scipy.linalg import sqrtm

def compute_svd(vals, k):
  # The magic happens here. U and V are user and item features
  U, s, V=np.linalg.svd(vals, full_matrices=False)
  s=np.diag(s)
  # we take only the k most significant features
  s=s[0:k,0:k]
  U=U[:,0:k]
  V=V[0:k,:]
  s_root=sqrtm(s)
  Usk = np.dot(U,s_root)
  skV = np.dot(s_root,V)
  UsV = np.dot(Usk, skV)

  return UsV

In [45]:

from sklearn.metrics import mean_squared_error

for k in range(2, len(effect_vals[0])):
  pred = compute_svd(effect_vals, k)

  print("k: ", k, " RMSE: ", mean_squared_error(effect_df, pred))

# for i in range(len(effect_vals)):
#   print("User ", i)
#   print(effect_vals[i])
#   print(pred[i].round())
#   print(effect_vals[i] - pred[i].round())



k:  2  RMSE:  1.2411394057543548
k:  3  RMSE:  1.0448868231714885
k:  4  RMSE:  0.859584740839658
k:  5  RMSE:  0.6913652961208363
k:  6  RMSE:  0.5666829435784825
k:  7  RMSE:  0.45479518487252946
k:  8  RMSE:  0.3486809065245669
k:  9  RMSE:  0.2618138692095433
k:  10  RMSE:  0.1765491842243626
k:  11  RMSE:  0.10162660795575242
k:  12  RMSE:  0.04373837283886026


In [50]:
meansures_vals = df[meansures_info_cols + measures_effect_info_cols].fillna(0).values

for k in range(2, len(meansures_vals[0])):
  pred = compute_svd(meansures_vals, k)

  print("k: ", k, " RMSE: ", mean_squared_error(meansures_vals, pred))

# for i in range(len(meansures_vals)):
#   print("User ", i)
#   print(meansures_vals[i])
#   print(pred[i].round())
#   print(meansures_vals[i] - pred[i].round())


k:  2  RMSE:  1.0074694617252216
k:  3  RMSE:  0.8432511931902505
k:  4  RMSE:  0.7180633103902063
k:  5  RMSE:  0.6037887087641539
k:  6  RMSE:  0.5150410467915527
k:  7  RMSE:  0.4364956112063118
k:  8  RMSE:  0.36129748764138664
k:  9  RMSE:  0.2895662947063093
k:  10  RMSE:  0.23123089496537283
k:  11  RMSE:  0.17469064780825772
k:  12  RMSE:  0.12354180161253538
k:  13  RMSE:  0.08489491200584477
k:  14  RMSE:  0.05538150103624959
k:  15  RMSE:  0.04132012895034858
k:  16  RMSE:  0.0311713369706
k:  17  RMSE:  0.025331678178247448
k:  18  RMSE:  0.02058944560075449
k:  19  RMSE:  0.016117174684000733
k:  20  RMSE:  0.012752622426227088
k:  21  RMSE:  0.009622866502087216
k:  22  RMSE:  0.006777782890855464
k:  23  RMSE:  0.0040689537384328385
k:  24  RMSE:  0.0018646737397042242
k:  25  RMSE:  0.000416706381627272
k:  26  RMSE:  5.732524623562827e-28
k:  27  RMSE:  5.733012680240592e-28
k:  28  RMSE:  5.733504098045648e-28
k:  29  RMSE:  5.733880754451e-28
k:  30  RMSE:  5.7343827

In [37]:
df[farm_info_cols]

,total_agr_land,workforce,trees,arable_land,ley,perm_grass,perm_crops,prot_cult,add_agr_land,oth_land,qual_landscape,eco_proof,organic,age
0,4925.0,1.95406,79,3871,107,793,140,0,0,165,0,1.0,0.0,62.0
1,2176.0,6.61310,12,1671,0,98,338,119,0,93,0,1.0,0.0,53.0
2,2251.0,1.39384,31,1507,279,380,81,0,4,512,0,1.0,0.0,60.0
3,2389.0,1.99523,3,1969,71,322,0,0,27,350,0,1.0,0.0,62.0
4,3205.0,3.09942,39,1911,675,461,117,0,14,112,0,1.0,0.0,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,1786.0,0.86598,32,719,646,167,0,0,0,23,0,1.0,0.0,58.0
101,876.0,4.29667,0,0,227,260,389,0,0,600,0,0.0,0.0,50.0
102,2995.0,2.96772,64,879,987,1115,0,0,14,474,0,1.0,0.0,63.0
103,6520.0,4.04098,0,4720,374,1385,0,0,41,1273,0,1.0,0.0,NaN
